In [27]:
import pickle
import glob
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import log_loss
import xgboost as xgb

# load feats
train_x,test_x = [],[]
for feat in sorted(glob.glob('../features/*.pkl')):
#     if 'tfidf' in feat or 'lr' in feat or 'mnb' in feat:
#         continue
    print('file path',feat)
    a,b = pickle.load(open(feat,'rb'))
    print(a.shape,b.shape)
    train_x.append(a)
    test_x.append(b)
train_x = np.nan_to_num(np.hstack(train_x))
test_x = np.nan_to_num(np.hstack(test_x))
print(train_x.shape)
    
# load y
train = pd.read_csv("../input/train.csv")
list_classes = ["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]
train_y = train[list_classes].values.astype('int')
print(train_x.shape)

file path ../features/cnn2d_muse_1_feat.pkl
(159571, 6) (153164, 6)
file path ../features/cnn_glove_1_feat.pkl
(159571, 6) (153164, 6)
file path ../features/cnn_gru_glove_1_feat.pkl
(159571, 6) (153164, 6)
file path ../features/cnn_muse_adj_1_feat_de_fr.pkl
(159571, 6) (153164, 6)
file path ../features/cudnn_gru_glove_1_feat.pkl
(159571, 6) (153164, 6)
file path ../features/lr_feat1.pkl
(159571, 6) (153164, 6)
file path ../features/lr_feat2.pkl
(159571, 6) (153164, 6)
file path ../features/lstm_glove_1_feat.pkl
(159571, 6) (153164, 6)
file path ../features/mnb_feat1.pkl
(159571, 6) (153164, 6)
file path ../features/mnb_feat2.pkl
(159571, 6) (153164, 6)
file path ../features/other_feat.pkl
(159571, 16) (153164, 16)
file path ../features/tfidf_feat1.pkl
(159571, 30) (153164, 30)
file path ../features/tfidf_feat2.pkl
(159571, 30) (153164, 30)
(159571, 136)
(159571, 136)


In [28]:
train[list_classes].describe()

,toxic,severe_toxic,obscene,threat,insult,identity_hate
count,159571.000000,159571.000000,159571.000000,159571.000000,159571.000000,159571.000000
mean,0.095844,0.009996,0.052948,0.002996,0.049364,0.008805
std,0.294379,0.099477,0.223931,0.054650,0.216627,0.093420
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [29]:
def aug_data_ratio(x,y,ratio=0.2):
    print(x.shape)
    neg_index = np.where(y[:,0]==0)[0]
    print(neg_index)
    data_cnt = len(neg_index)
    add_cnt = int(data_cnt*ratio)
    add_index = neg_index[:add_cnt]
    add_x = np.concatenate([x,x[add_index]])
    add_y = np.concatenate([y,y[add_index]])
    print(add_x.shape,data_cnt)
    return add_x,add_y

def del_data_ratio(x,y,ratio=0.8):
    print(x.shape)
    pos_index = np.where(y[:,0]==1)[0]
    neg_index = np.where(y[:,0]==0)[0]
    print(pos_index)
    data_cnt = len(pos_index)
    add_cnt = int(data_cnt*ratio)
    add_index = pos_index[:add_cnt]
    add_x = np.concatenate([x[add_index],x[neg_index]])
    add_y = np.concatenate([y[add_index],y[neg_index]])
    print(add_x.shape,data_cnt)
    return add_x,add_y

# add neg data
#train_x,train_y = aug_data_ratio(train_x,train_y)
# del pos data
train_x,train_y = del_data_ratio(train_x,train_y)

(159571, 136)
[     6     12     16 ..., 159541 159546 159554]
(156512, 136) 15294


In [30]:
from sklearn.model_selection import KFold
def simple_ens(model_name,k=3,rnd=233):
    kf = KFold(n_splits=k, shuffle=True, random_state=rnd)
    test_pred = np.zeros((153164,6))
    cache_test_pred = np.zeros((153164,6))
    single_best = 100
    single_best_pred = None
    all_train_loss_l,all_val_loss_l = 0,0
    
    for train_index, test_index in kf.split(train_x):
        # x,y
        curr_x,curr_y = train_x[train_index],train_y[train_index]
        hold_out_x,hold_out_y = train_x[test_index],train_y[test_index]
        val_loss_l,train_loss_l = 0,0
        d_test = xgb.DMatrix(test_x)
        
        # share params
        params = {
                'colsample_bytree': 0.9,
                'subsample': 0.9,
                'eta': 0.05,
                'max_depth': 3,
                'eval_metric':'logloss',
                'objective':'binary:logistic',
                'scale_pos_weight':0.9,
                'colsample_bylevel':0.9,
                'colsample_bytree':0.9,
            
                }
        
        # train for each class
        for i in range(6):
            d_train = xgb.DMatrix(curr_x, curr_y[:,i])
            d_valid = xgb.DMatrix(hold_out_x, hold_out_y[:,i])
            watchlist = [(d_train, 'train'), (d_valid, 'valid')]

            model = xgb.train(params, d_train, 1000, watchlist,
                              early_stopping_rounds=50,
                              verbose_eval=2000)
            print(i)
            try:
                curr_train_loss = log_loss(curr_y[:,i],model.predict(d_train))
                curr_val_loss = log_loss(hold_out_y[:,i],model.predict(d_valid))
                print(curr_train_loss,curr_val_loss)
                val_loss_l += curr_val_loss
                train_loss_l += curr_train_loss
            except:
                pass
            curr_test_pred = model.predict(d_test)
            
            test_pred[:,i] += curr_test_pred
            cache_test_pred[:,i] += curr_test_pred
            
        # avg 6 class
        train_loss_l = train_loss_l/6
        val_loss_l = val_loss_l/6
        print('this fold avg train',train_loss_l,'avg val',val_loss_l)
        
        # save best one fold result
        if val_loss_l < single_best:
            single_best = val_loss_l
            single_best_pred = cache_test_pred
            print('new single best')
        
        cache_test_pred = np.zeros((153164,6))
        
        # avg k fold
        all_train_loss_l += train_loss_l/k
        all_val_loss_l += val_loss_l/k
        print('========================')
    test_pred = test_pred/k
    print('all train avg',all_train_loss_l,'all val avg',all_val_loss_l)
    return test_pred, single_best_pred

print('done')

done


In [31]:
xgb_res,b = simple_ens('xgb',k=3)
sample_submission = pd.read_csv("../input/sample_submission.csv")
sample_submission[list_classes] = xgb_res
sample_submission.to_csv("../results/xgb_ens_sample_csv_fold3.gz", index=False, compression='gzip')
print(sample_submission.head())
print('save done')


[0]	train-logloss:0.648657	valid-logloss:0.64872
Multiple eval metrics have been passed: 'valid-logloss' will be used for early stopping.

Will train until valid-logloss hasn't improved in 50 rounds.
Stopping. Best iteration:
[409]	train-logloss:0.062519	valid-logloss:0.072111

0
0.0614208203263 0.0721179635368
[0]	train-logloss:0.645433	valid-logloss:0.645393
Multiple eval metrics have been passed: 'valid-logloss' will be used for early stopping.

Will train until valid-logloss hasn't improved in 50 rounds.
Stopping. Best iteration:
[186]	train-logloss:0.014686	valid-logloss:0.016606

1
0.0136863819794 0.0166844628059
[0]	train-logloss:0.646561	valid-logloss:0.646584
Multiple eval metrics have been passed: 'valid-logloss' will be used for early stopping.

Will train until valid-logloss hasn't improved in 50 rounds.
Stopping. Best iteration:
[252]	train-logloss:0.031944	valid-logloss:0.036799

2
0.0308156101461 0.0368382811667
[0]	train-logloss:0.644687	valid-logloss:0.644722
Multiple 

In [33]:
sample_submission = pd.read_csv("../input/sample_submission.csv")
sample_submission[list_classes] = xgb_res/1.2
sample_submission.to_csv("../results/xgb_ens_sample_csv_fold3_div2.gz", index=False, compression='gzip')
print(sample_submission.head())
print('save done')

                 id     toxic  severe_toxic   obscene    threat    insult  \
0  00001cee341fdb12  0.831418      0.242548  0.814362  0.104252  0.776158   
1  0000247867823ef7  0.000082      0.000022  0.000070  0.000017  0.000061   
2  00013b17ad220c46  0.000478      0.000024  0.000329  0.000022  0.000315   
3  00017563c3f7919a  0.000081      0.000024  0.000099  0.000023  0.000083   
4  00017695ad8997eb  0.002487      0.000021  0.000470  0.000039  0.000211   

   identity_hate  
0       0.438007  
1       0.000036  
2       0.000136  
3       0.000051  
4       0.000110  
save done


In [34]:
xgb_res,b = simple_ens('xgb',k=5)
sample_submission = pd.read_csv("../input/sample_submission.csv")
sample_submission[list_classes] = xgb_res/1.2
sample_submission.to_csv("../results/xgb_ens_sample_csv_fold5_div2.gz", index=False, compression='gzip')
print(sample_submission.head())
print('save done')


[0]	train-logloss:0.648634	valid-logloss:0.648767
Multiple eval metrics have been passed: 'valid-logloss' will be used for early stopping.

Will train until valid-logloss hasn't improved in 50 rounds.
Stopping. Best iteration:
[612]	train-logloss:0.059786	valid-logloss:0.07258

0
0.0589449559538 0.0726483673998
[0]	train-logloss:0.645423	valid-logloss:0.645405
Multiple eval metrics have been passed: 'valid-logloss' will be used for early stopping.

Will train until valid-logloss hasn't improved in 50 rounds.
Stopping. Best iteration:
[183]	train-logloss:0.014843	valid-logloss:0.016856

1
0.0140196628997 0.0168684381444
[0]	train-logloss:0.646537	valid-logloss:0.646585
Multiple eval metrics have been passed: 'valid-logloss' will be used for early stopping.

Will train until valid-logloss hasn't improved in 50 rounds.
Stopping. Best iteration:
[448]	train-logloss:0.029475	valid-logloss:0.035661

2
0.0287347146993 0.0356924805198
[0]	train-logloss:0.644699	valid-logloss:0.644724
Multiple 

[0]	train-logloss:0.645407	valid-logloss:0.645474
Multiple eval metrics have been passed: 'valid-logloss' will be used for early stopping.

Will train until valid-logloss hasn't improved in 50 rounds.
Stopping. Best iteration:
[222]	train-logloss:0.014083	valid-logloss:0.017466

1
0.013270601785 0.0175445806101
[0]	train-logloss:0.646537	valid-logloss:0.646495
Multiple eval metrics have been passed: 'valid-logloss' will be used for early stopping.

Will train until valid-logloss hasn't improved in 50 rounds.
Stopping. Best iteration:
[275]	train-logloss:0.032462	valid-logloss:0.035738

2
0.0315677244849 0.035760534721
[0]	train-logloss:0.644694	valid-logloss:0.644726
Multiple eval metrics have been passed: 'valid-logloss' will be used for early stopping.

Will train until valid-logloss hasn't improved in 50 rounds.
Stopping. Best iteration:
[205]	train-logloss:0.003966	valid-logloss:0.00648

3
0.00340480161608 0.00648508137196
[0]	train-logloss:0.647246	valid-logloss:0.647329
Multiple 